## Data Merging and Pivot Table Analysis

In [1]:
import pandas as pd

xls = pd.ExcelFile('yelp.xlsx')
df = xls.parse('yelp_data')

print(df.head())

                           name   category_0       category_1  take_out  \
0  China Sea Chinese Restaurant  Restaurants          Chinese      True   
1          Discount Tire Center        Tires       Automotive     False   
2                  Frankfurters  Restaurants         Hot Dogs      True   
3             Fred Dietz Floral     Shopping  Flowers & Gifts     False   
4                 Kuhn's Market         Food          Grocery     False   

   review_count  stars  city_id  state_id  
0            11    2.5        1         1  
1            24    4.5        1         1  
2             3    4.5        1         1  
3             6    4.0        1         1  
4             8    3.5        1         1  


> ## Parsing excel worksheet
> ### parses the cities worksheet from the Excel file to own data frame

In [2]:
df_cities = xls.parse('cities')

print(df_cities.head())

   id           city
0   1       Bellevue
1   2       Braddock
2   3       Carnegie
3   4      Homestead
4   5  Mc Kees Rocks


>## # Join keys 
This code combines two DataFrames by matching rows where city_id in df equals id in df_cities using an inner join. It then prints the first few rows of the resulting merged DataFrame.

In [3]:
df = pd.merge(left=df, right=df_cities, how='inner', left_on='city_id', right_on='id')

print(df.head())

                           name   category_0       category_1  take_out  \
0  China Sea Chinese Restaurant  Restaurants          Chinese      True   
1          Discount Tire Center        Tires       Automotive     False   
2                  Frankfurters  Restaurants         Hot Dogs      True   
3             Fred Dietz Floral     Shopping  Flowers & Gifts     False   
4                 Kuhn's Market         Food          Grocery     False   

   review_count  stars  city_id  state_id  id      city  
0            11    2.5        1         1   1  Bellevue  
1            24    4.5        1         1   1  Bellevue  
2             3    4.5        1         1   1  Bellevue  
3             6    4.0        1         1   1  Bellevue  
4             8    3.5        1         1   1  Bellevue  


> ## Parse and merge
The purpose of the code is to combine data from two different tables into one. One table (df) might have information like transactions or details, and the other table (df_states) holds additional details about states from an Excel sheet. By matching the state ID in df with the ID in df_states, the code brings together related information. It also uses custom labels for any columns with the same name so that no conflicts occur when the two tables are merged.


In [4]:
# We take data from an Excel sheet called "states" and turn it into a table (or DataFrame) called df_states.
df_states = xls.parse('states')

# We combine our main table (df) with the df_states table. We match rows where the state_id in df is the same as the id in df_states.
# We use an "inner" join, which means only the rows that match in both tables will show up in the final result.
# We add custom labels (_left for df and _states for df_states) to any columns that have the same name in both tables.
df = pd.merge(
    left=df, 
    right=df_states, 
    how='inner', 
    left_on='state_id', 
    right_on='id',
    suffixes=('_left', '_states')
)

# we display the first few rows of the merged table to see what it looks like.
print(df.head())

                           name   category_0       category_1  take_out  \
0  China Sea Chinese Restaurant  Restaurants          Chinese      True   
1          Discount Tire Center        Tires       Automotive     False   
2                  Frankfurters  Restaurants         Hot Dogs      True   
3             Fred Dietz Floral     Shopping  Flowers & Gifts     False   
4                 Kuhn's Market         Food          Grocery     False   

   review_count  stars  city_id  state_id  id_left      city  id_states state  
0            11    2.5        1         1        1  Bellevue          1    PA  
1            24    4.5        1         1        1  Bellevue          1    PA  
2             3    4.5        1         1        1  Bellevue          1    PA  
3             6    4.0        1         1        1  Bellevue          1    PA  
4             8    3.5        1         1        1  Bellevue          1    PA  


## Aggregating Review Counts by State and City
This code creates a summary table from your DataFrame (df) by grouping data based on the columns "state_id" and "city_id". For each unique combination of these two columns, it adds up all the values in the "review_count" column. The result is stored in the variable pivot_agg, which you can later print or use for further analysis.

In [5]:
import pandas as pd

pivot_agg = pd.pivot_table(
    df,
    index=['state_id', 'city_id'],
    values=['review_count'],
    aggfunc="sum"  # using string instead of np.sum
)

print(pivot_agg)

                  review_count
state_id city_id              
1        1                 158
         2                  29
         3                 299
         4                 953
         5                 107
         6                  25
         7                  91
         8                6470
         9                 124
         10                 51
2        11               4332
         12               7226
         13                398


## Further segmenting using columns

The below code creates a pivot table that summarizes review counts by state and city, while also splitting the results based on whether an item is take-out. Here's what happens step-by-step:
- Grouping: The table is grouped by unique pairs of state_id and city_id.
- Aggregation: For each group, it sums up the values in the review_count column.
- Segmentation: The columns=['take_out'] parameter splits the results into separate columns based on the unique values of take_out (for example, True or False).
- Output: The resulting pivot table is printed, showing the aggregated review counts for each state-city combination, segmented by the take-out indicator.

In [6]:
pivot_agg2 = pd.pivot_table(
    df, index=['state_id', 'city_id'],
    values=['review_count'],  
    columns=['take_out'],  
    aggfunc="sum" 
)
print(pivot_agg2)

                 review_count      
take_out                False True 
state_id city_id                   
1        1                 52   106
         2                  3    26
         3                 74   225
         4                323   630
         5                 48    59
         6                 13    12
         7                 12    79
         8               1447  5023
         9                  7   117
         10                36    15
2        11              2009  2323
         12              1619  5607
         13               293   105


## Aggregating Review Counts and Average Ratings by Location and Take-Out Option

This code creates a summary table that breaks down your data in two ways:

- Grouping: It groups the data by "state" and "city," so each row in the output represents a unique state-city pair.
- Segmenting by Take-Out: It then separates the results into different columns based on the "take_out" values (for example, True/False).
- Aggregating Data: For each state-city group and take-out category, it:
- Sums up all the "review_count" values.
- Calculates the average ("mean") of the "stars" ratings.
- Prints out this pivot table to show you the aggregated review counts and average ratings for each combination.


In [7]:
pivot_agg3 = pd.pivot_table(
    df,
    index=['state_id', 'city_id'],
    columns=['take_out'],
    aggfunc={'review_count': 'sum', 'stars': 'mean'}
)
print(pivot_agg3)

                 review_count           stars          
take_out                False True      False     True 
state_id city_id                                       
1        1                 52   106  3.750000  3.750000
         2                  3    26  5.000000  4.500000
         3                 74   225  3.269231  3.722222
         4                323   630  3.416667  2.909091
         5                 48    59  3.666667  3.714286
         6                 13    12  3.000000  3.500000
         7                 12    79  3.000000  3.000000
         8               1447  5023  3.836364  3.566265
         9                  7   117  4.500000  2.750000
         10                36    15  3.500000  4.333333
2        11              2009  2323  3.469697  3.258065
         12              1619  5607  3.423077  3.310345
         13               293   105  3.155172  2.562500


# Problem #1: Explore the Data and Create a Merged Dataset
Problem 1: Explore the Data and Create a Merged Dataset
> Task: You are given Yelp business data, along with city and state information. 
- Your first task is to explore the data, understand its structure, and then merge the relevant datasets to get a more comprehensive view of the businesses.

> Instructions:
- Examine the yelp_data, cities, and states datasets. What columns or attributes from each dataset are relevant for understanding a business in terms of location and category?
- Write a program that merges the yelp_data with the cities and states datasets using appropriate keys. Be sure to include columns such as business name, category, city, state, and rating.
- How does the city size or type of businesses might influence the ratings. Does the data give you any insights on this?

In [ ]:
# Write your code here






# Problem 2: Investigating Average Ratings by Location
>Task: After merging the datasets, you need to analyze the average ratings of businesses in different cities.

> Instructions:
- Using the merged dataset, decide how to group the data. What makes sense for comparing ratings across locations?
- Compute the average rating (stars) for each city, and determine the city with the highest and lowest average ratings.
- Reflect on how city size or type of businesses might influence the ratings. Does the data give you any insights on this?

In [ ]:
# Write your code here






# Problem 3: Business Category Analysis Across States
>Task: Now that you have the merged data, your task is to analyze the distribution of business categories and their ratings across different states.

> Instructions:

- Consider how you can group businesses by state and category, and what kind of summary you want to generate.
- Create a pivot table showing the average rating (stars) for each business category by state. What do you notice in the data? Are some states dominated by certain types of businesses or have higher ratings in specific categories?
- Based on your pivot table, what interesting trends or patterns can you conclude about the business landscape across states?

In [ ]:
# Write your code here



